In [7]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import scipy.stats as stats
import statsmodels.formula.api as smf
from scipy.stats import bernoulli
from scipy.stats import truncnorm
import math

In [8]:
def DGP(intervene_A=None):
    p = 0.6     # Above, this was defined as 0.5. In paper, it's 0.6
    n = 10000

    # A (gender)
    if intervene_A:
        A2 = np.ones(n)
    else:
        A2 = U_A = bernoulli.rvs(p=p, size=n)


    # Q (qualifications)
    U_Q = np.random.normal(2, 5, n)
    Q2 = np.floor(U_Q)


    # D (number of children)
    lower_d, upper_d = 0.1, 3
    mu_d, sigma_d = 2, 1
    X_d = stats.truncnorm((lower_d - mu_d) / sigma_d, (upper_d - mu_d) / sigma_d, loc=mu_d, scale=sigma_d)
    U_D = X_d.rvs(n)
    D2 = A2 + np.floor(0.5 * Q2 * U_D)


    # M (physical strength)
    lower_m, upper_m = 0.1, 3
    mu_m, sigma_m = 3, 2
    X_m = stats.truncnorm((lower_m - mu_m) / sigma_m, (upper_m - mu_m) / sigma_m, loc=mu_m, scale=sigma_m)
    U_M = X_m.rvs(n)
    M2 = 3*A2 + (0.4 * Q2 * U_M)


    sigmoid = lambda x: 1/(1+ math.exp(-x))

    Y2 = np.zeros(n)
    for i in range(n):
        Y2[i] = sigmoid(-10+5*A2[i]+Q2[i]+D2[i]+M2[i]) >= 0.5


    df = pd.DataFrame({'A': A2, 'D': D2, 'M': M2 ,'Q': Q2,'Y':Y2})
    
    return df

In [9]:
#counterfactual gender
def counterfactual_df(without_intervention):
    n = 10000
    A2 = 1 - without_intervention.A.values

    # Q (qualifications) - unchanged
    Q2 = without_intervention.Q.values


    # D (number of children)
    lower_d, upper_d = 0.1, 3
    mu_d, sigma_d = 2, 1
    X_d = stats.truncnorm((lower_d - mu_d) / sigma_d, (upper_d - mu_d) / sigma_d, loc=mu_d, scale=sigma_d)
    U_D = X_d.rvs(n)
    D2 = A2 + np.floor(0.5 * Q2 * U_D)


    # M (physical strength)
    lower_m, upper_m = 0.1, 3
    mu_m, sigma_m = 3, 2
    X_m = stats.truncnorm((lower_m - mu_m) / sigma_m, (upper_m - mu_m) / sigma_m, loc=mu_m, scale=sigma_m)
    U_M = X_m.rvs(n)
    M2 = 3*A2 + (0.4 * Q2 * U_M)


    sigmoid = lambda x: 1/(1+ math.exp(-x))

    Y2 = np.zeros(n)
    for i in range(n):
        Y2[i] = sigmoid(-10+5*A2[i]+Q2[i]+D2[i]+M2[i]) >= 0.5

    Counterfactual_df = pd.DataFrame({'A': A2, 'D': D2, 'M': M2 ,'Q': Q2,'Y':Y2})
    return Counterfactual_df

In [17]:
#Data Preperation
def data_processing(without_intervention, Counterfactual_df):
    from sklearn.model_selection import train_test_split
    #discretize fair fair Q
    without_intervention['bi_Q'] = np.where(without_intervention['Q'] <= np.quantile(without_intervention['Q'], 0.5), -1, 1)
    Counterfactual_df['bi_Q'] = np.where(Counterfactual_df['Q'] <= np.quantile(Counterfactual_df['Q'], 0.5), -1, 1)
    
    X_train, X_test, y_train, y_test = train_test_split(without_intervention.drop(['Y'], axis = 1), 
                                                        without_intervention.Y,
                                                        test_size = 0.2, 
                                                        stratify = without_intervention.Y)
    #get the same rows of counterfactual test data
    Counter_X_test = Counterfactual_df.drop(['Y'], axis = 1).iloc[X_test.index.values, :]
    Counter_y_test = Counterfactual_df['Y'][X_test.index.values]
    
    return X_train, X_test, y_train, y_test, Counter_X_test, Counter_y_test


In [12]:
#Model training and prediction results:
def trainers(X_train, y_train, full_cols, unaware_cols, fair_var, model):
    from sklearn.linear_model import LogisticRegression
    from sklearn.tree import DecisionTreeClassifier
    from sklearn import svm
    
    if model == 'log':
        full = LogisticRegression()
        unaware = LogisticRegression()
        fair = LogisticRegression()
        
        full.fit(X_train[full_cols], y_train)
        unaware.fit(X_train[unaware_cols], y_train)
        fair.fit(X_train[fair_var], y_train)
    
    elif model == 'svm':
        full = svm()
        unaware = svm()
        fair = svm()
        
        full.fit(X_train[full_cols], y_train)
        unaware.fit(X_train[unaware_cols], y_train)
        fair.fit(X_train[fair_var], y_train)
        
        
    elif model == 'dt':
        full = DecisionTreeClassifier()
        unaware = DecisionTreeClassifier()
        fair = DecisionTreeClassifier()
        
        full.fit(X_train[full_cols], y_train)
        unaware.fit(X_train[unaware_cols], y_train)
        fair.fit(X_train[fair_var], y_train)
    
    else:
        dec = 'model is not included in is trainer'
        return dec
    
    return {'full': full, 'unaware': unaware, 'fair': fair}

In [19]:
#get predictions
def get_predictions(X_test, Counter_X_test, full_cols, unaware_cols, fair_var, fitted_models):
    import pandas as pd
    
    predictions = {}
    counter_predictions = {}
    
    for model in fitted_models:
        if model == 'full':
            predictions[model] = fitted_models[model].predict(X_test[full_cols])
            counter_predictions[model] = fitted_models[model].predict(Counter_X_test[full_cols])
        if model == 'unaware':
            predictions[model] = fitted_models[model].predict(X_test[unaware_cols])
            counter_predictions[model] = fitted_models[model].predict(Counter_X_test[unaware_cols])
        if model == 'fair':
            predictions[model] = fitted_models[model].predict(X_test[fair_var])
            counter_predictions[model] = fitted_models[model].predict(Counter_X_test[fair_var])

    return pd.DataFrame(predictions), pd.DataFrame(counter_predictions)

In [21]:
## counterfactual fairness: sum of absulute difference (counter yhat - factual yhat)
def counterfactual_fairness(counter_prediction, factual_prediction):
    return sum(abs(counter_prediction - factual_prediction))/len(counter_prediction)

## ETT fairness: absulute value difference between sum of counter predicted y and fautual predicted y
def ett_fairness(counter_prediction, factual_prediction):
    return abs(sum(factual_prediction)- sum(counter_prediction))/len(X_test)

In [20]:
without_intervention = DGP(intervene_A=None)
Counterfactual_df = counterfactual_df(without_intervention)
X_train, X_test, y_train, y_test, Counter_X_test, Counter_y_test = data_processing(without_intervention, Counterfactual_df)

full_cols = ['A', 'Q', 'D', 'M']
unaware_cols = ['Q', 'D', 'M']
fair_var = ['Q']
model = 'log'

fitted_models = trainers(X_train, y_train, full_cols, unaware_cols, fair_var, model)
pred, counter_pred = get_predictions(X_test, Counter_X_test, full_cols, unaware_cols, fair_var, fitted_models)

In [27]:
pd.DataFrame({
    'ETT': [
        ett_fairness(pred['full'], counter_pred['full']),
        ett_fairness(pred['unaware'], counter_pred['unaware']),
        ett_fairness(pred['fair'], counter_pred['fair'])
    ],
    'Counterfactual':[
        counterfactual_fairness(pred['full'], counter_pred['full']),
        counterfactual_fairness(pred['unaware'], counter_pred['unaware']),
        counterfactual_fairness(pred['fair'], counter_pred['fair'])
    ],
    'Accuracy':[
        fitted_models['full'].score(X_test[full_cols], y_test),
        fitted_models['unaware'].score(X_test[unaware_cols], y_test),
        fitted_models['fair'].score(X_test[fair_var], y_test)
        
    ]
}, index = ['Full Model', 'Unaware Model', 'Fair Predictor Q'])

,ETT,Counterfactual,Accuracy
Full Model,0.039,0.272,0.9955
Unaware Model,0.036,0.234,0.9730
Fair Predictor Q,0.000,0.000,0.8670
